<a href="https://colab.research.google.com/github/cdieng0/projet-stattapp/blob/main/ragrse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymupdf
!pip install faiss-cpu
!pip install sentence_transformers
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

# Nouvelle section

In [2]:
import os
import fitz  # PyMuPDF pour l'extraction de texte et d'images
import numpy as np
import faiss
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import sent_tokenize
# Import pour le NLP
from sentence_transformers import SentenceTransformer, CrossEncoder,util
from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import re
from huggingface_hub import login

In [3]:
# from huggingface_hub import login

# login(token="")  # Récupérez votre token ici : https://huggingface.co/settings/tokens

# generator = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")

In [4]:
def extract_text_from_pdf(pdf_path):
    """
    Extrait le texte brut d’un fichier PDF.

    Args:
        pdf_path (str): Chemin vers le fichier PDF.

    Returns:
        str: Texte complet extrait du PDF, concaténé page par page avec des sauts de ligne.
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text() + "\n"
    return text


def chunk_text(text):
    """
    Découpe le texte en morceaux (chunks) de taille fixe avec recouvrement (overlap).

    Args:
        text (str): Texte complet à découper.

    Returns:
        list: Liste de chaînes de caractères, chaque chunk contenant environ 300 mots
              avec 30 mots de chevauchement pour préserver le contexte.
    """
    chunk_size, overlap = 300, 30
    words = text.split()
    chunks = []
    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = words[i:i+chunk_size]
        chunks.append(" ".join(chunk))
    return chunks


def encode_passages(passages, model):
    """
    Encode chaque passage textuel en vecteur d’embedding avec un modèle de type SentenceTransformer.

    Args:
        passages (list): Liste de chaînes de caractères à encoder.
        model (SentenceTransformer): Modèle d'embedding, par exemple E5 ou MiniLM.

    Returns:
        np.ndarray: Matrice d’embeddings de type float32 (shape: [n_passages, embedding_dim]).
    """
    embeddings = model.encode(passages, show_progress_bar=True)
    return np.array(embeddings, dtype=np.float32)


def build_faiss_index(embeddings):
    """
    Construit un index FAISS à partir des embeddings (utilise la distance L2).

    Args:
        embeddings (np.ndarray): Matrice des embeddings à indexer (shape: [n, d]).

    Returns:
        faiss.IndexFlatL2: Index FAISS entraîné et prêt pour les requêtes.
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index


def retrieve_passages(query, passages, embedding_model, faiss_index, k=10):
    """
    Recherche les k passages les plus pertinents pour une requête donnée à l'aide de FAISS.

    Args:
        query (str): Question ou phrase à rechercher.
        passages (list): Liste des passages (chunks) d’origine.
        embedding_model (SentenceTransformer): Modèle d’embedding utilisé pour encoder la requête.
        faiss_index (faiss.IndexFlatL2): Index FAISS contenant les embeddings des passages.
        k (int): Nombre de résultats à retourner.

    Returns:
        list: Liste des k passages les plus proches (par similarité).
    """
    query_embedding = embedding_model.encode([query])
    distances, indices = faiss_index.search(np.array(query_embedding, dtype=np.float32), k)
    retrieved = [passages[idx] for idx in indices[0] if idx < len(passages)]
    return retrieved


def rerank_passages(query, passages, reranker, threshold=0.4):
    """
    Recalcule la pertinence des passages récupérés avec un modèle Cross-Encoder type MiniLM,
    puis filtre et trie les passages les plus pertinents.

    Args:
        query (str): Question initiale de l’utilisateur.
        passages (list): Passages récupérés initialement via FAISS.
        reranker (CrossEncoder): Modèle Cross-Encoder (type ms-marco) pour scoring binaire.
        threshold (float): Seuil minimal pour conserver un passage.

    Returns:
        list of tuples: Liste de (passage, score) triés par score décroissant.
    """
    pairs = [(query, passage) for passage in passages]
    scores = reranker.predict(pairs)
    filtered = [(passage, score) for passage, score in zip(passages, scores) if score > threshold]
    ranked = sorted(filtered, key=lambda x: x[1], reverse=True)
    return ranked


def generate_answer(query, ranked_passages, max_length=500):
    """
    Génère une réponse en paragraphe à partir des passages rerankés.

    Args:
        query (str): La question posée
        ranked_passages (list): Liste de tuples (passage, score)
        max_length (int): Longueur maximale de la réponse

    Returns:
        str: Réponse formatée en paragraphe
    """
     # 2. Créer le contexte pour le prompt
    context = "\\n".join([f"[Source {i+1}] {text}"  for i, text in enumerate(ranked_passages)])  # Prendre les passages rerankes

    # 3. Construire le prompt instruct
    prompt = f"""<s>[INST]
    En tant qu'expert en RSE, synthétisez une réponse précise en français en utilisant UNIQUEMENT
    les sources fournies. Structurez la réponse en un paragraphe de 400 mots avec des points clés.

    Question: {query}

    Sources:
    {context}

    Répondez dans un français clair et concis: [/INST]"""

    # 4. Génération avec Mistral (version ouverte alternative)


    response = generator(
        prompt,
        max_new_tokens=max_length,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1
    )

    return response[0]['generated_text'].split("[/INST]")[-1].strip()


def find_top_chunks_for_phrase(phrase, chunks, top_k=4, threshold=0.5):
    """
    Trouve les top_k chunks les plus similaires à une phrase pertinente.

    Args:
        phrase (str): La phrase de référence.
        chunks (List[str]): Tous les chunks du document.
        top_k (int): Nombre de passages à retourner.
        threshold (float): Score de similarité minimale.

    Returns:
        List[Tuple[int, float, str]]: (chunk_id, score, texte) triés par pertinence.
    """
    # Encodage
    phrase_embedding = embedding_model.encode(f"query: {phrase}", convert_to_tensor=True)
    chunk_texts = [f"passage: {chunk}" for chunk in chunks]
    chunk_embeddings = embedding_model.encode(chunk_texts, convert_to_tensor=True)

    # Similarité cosinus
    similarities = util.cos_sim(phrase_embedding, chunk_embeddings)[0]

    # Tri décroissant
    top_indices = torch.argsort(similarities, descending=True)

    # Récupération des top chunks au-dessus du seuil
    top_matches = []
    for idx in top_indices[:top_k * 2]:  # on élargit au cas où certains soient sous le seuil
        score = similarities[idx].item()
        if score >= threshold:
            top_matches.append((idx.item(), score, chunks[idx.item()]))
        if len(top_matches) == top_k:
            break

    return top_matches

def calculate_mrr(questions_data, retrieved_passages_by_question, threshold=0.85):
    """
    Calcule le MRR en comparant uniquement le premier passage du ground truth
    avec les chunks retrouvés (et non tous les chunks du corpus !).

    Args:
        questions_data (list): liste de questions + ground truth
            [
                {
                    "question": "...",
                    "relevant_chunks": ["..."]  # on prend le premier
                },
                ...
            ]
        retrieved_passages_by_question (dict): {question: [retrieved_chunks (texte)]}
        threshold (float): seuil de similarité cosinus pour considérer un match

    Returns:
        float: MRR global
    """
    reciprocal_ranks = []

    for q_data in questions_data:
        question = q_data["question"]
        ground_truth_chunk = q_data["relevant_chunks"][0]  # on compare au 1er chunk GT

        # Embedding du ground truth
        gt_embedding = model.encode(f"passage: {ground_truth_chunk}", convert_to_tensor=True)

        # Passages récupérés uniquement pour cette question
        retrieved_passages = retrieved_passages_by_question.get(question, [])

        found = False

        for rank, passage in enumerate(retrieved_passages, start=1):
            passage_embedding = model.encode(f"passage: {passage}", convert_to_tensor=True)
            sim = util.cos_sim(gt_embedding, passage_embedding)[0][0].item()

            if sim >= threshold:
                reciprocal_ranks.append(1 / rank)
                found = True
                break  # stop at first match

        if not found:
            reciprocal_ranks.append(0.0)

    return sum(reciprocal_ranks) / len(reciprocal_ranks) if reciprocal_ranks else 0.0

In [7]:
pdf_file = os.listdir("data")

# Nouvelle section

In [8]:
pdf_file

['eib-group-2022-climate-bank-roadmap-progress-report.pdf',
 '20240903-bnpparibas-csr-presentation-2024.pdf',
 'climate-change-and-the-banking-industry.pdf',
 'as_101_climate_risk_banks_en.pdf',
 'What-contribution-do-banks-make-to-the-ecological-transition.pdf',
 'bnp_paribas_2023_climate_report.pdf',
 'ssm.202011finalguideonclimate-relatedandenvironmentalrisks~58213f6564.fr.pdf']

In [21]:
full_text=""
for doc in pdf_file:
    pdf_path="data/"+doc
    full_text+=" "+ extract_text_from_pdf(pdf_path)

In [22]:
# taille avant ajout des images
len(full_text)

752720

importation des description images

In [11]:
data=pd.read_csv("/content/caption_image.csv")

In [18]:
caption=data["caption"]

Fusion avec les la liste des texts extraits

In [23]:
for el in caption:
  full_text+=" "+el

In [24]:
# taille apres ajout des images
len(full_text)

768383

### Initialisation du model

In [25]:
 #  Extraction du texte et découpage en passages
 passages = chunk_text(full_text)
 print(f"Nombre de passages extraits: {len(passages)}")

#  Chargement du embeddings e5
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")
passage_embeddings = encode_passages(passages, embedding_model)
faiss_index = build_faiss_index(passage_embeddings)

Nombre de passages extraits: 419


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
if __name__ == "__main__":

    # Définir une requête utilisateur
    query = "Quel est l’objectif de BNP Paribas pour les prêts durables d’ici 2025 ?"#"Quelles entreprises ont réduit leurs émissions de CO2 de manière significative ?"

    #  Recherche initiale via FAISS
    retrieved_passages = retrieve_passages(query, passages, embedding_model, faiss_index, k=10)
    print("\nPassages récupérés (avant reranking) :")
    for p in retrieved_passages:
        print(" -", p[:100], "...")

    # #  Reranking avec MiniLM Cross-Encoder
    # reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    # ranked_passages = rerank_passages(query, retrieved_passages, reranker, threshold=0.5)

    # print("\nPassages après reranking et filtrage par seuil de pertinence :")
    # for i, (passage, score) in enumerate(ranked_passages, 1):
    #     print(f"{i}. (Score: {score:.2f}) {passage[:150]}...")

    #Generation
    # reponse = generate_answer(query, ranked_passages)
    # print("Réponse générée:\\n", reponse)

        reponse = generate_answer(query, retrieved_passages)
    print("Réponse générée:\\n", reponse)

    print("\n Ta rag RAG exécuté avec succès.")

Nombre de passages extraits: 16


Batches: 100%|██████████| 1/1 [00:34<00:00, 34.63s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Passages récupérés (avant reranking) :
 - of the Group’s CSR policy, impacting the annual variable compensation applicable to the CEO and the  ...



KeyboardInterrupt



In [ ]:
#  Le vrai ground truth (phrases pertinentes extraites manuellement)
questions_data = [
    {
        "question": "Quel est l’objectif de BNP Paribas pour les prêts durables d’ici 2025 ?",
        "relevant_phrases": [
            "BNP Paribas vise un montant de prêts durables de 150 milliards d’euros d’ici 2025 (contre 117 milliards en 2023).",
            "Le groupe a réduit de 70% ses financements aux énergies fossiles depuis 2020.",
            "BNP Paribas est classé #1 mondial en finance durable en 2023."
        ]
    }
]

retrieved_ids = [passages.index(p) for p in retrieved_passages]

# 3. Structure pour le calcul MRR
retrieved_chunks_by_question = {
    "Quel est l’objectif de BNP Paribas pour les prêts durables d’ici 2025 ?": retrieved_ids
}

# Calcul du MRR
mrr = calculate_mrr(questions_data,all_chunks=passages, retrieved_chunks_by_question=retrieved_chunks_by_question, threshold=0.85
)

print(f" MRR: {mrr:.3f}")


### Great Truth pour dautre test mrr

##### On remplace query par la question et cesT OK

In [ ]:
questions_data=[ {
        "question": "Quels sont les engagements climatiques de BNP Paribas pour 2050 ?",
        "relevant_phrases": [
            "BNP Paribas s’est engagée à atteindre la neutralité carbone d’ici 2050.",
            "La banque aligne ses portefeuilles sur les scénarios de l’AIE."
        ]
    }
               ]

questions_data=[  {
        "question": "How does the EIB support adaptation to climate change in the European Union and beyond?",
        "relevant_phrases": [
            "In 2022 the EIB lent €1.8 billion for climate change adaptation, of which nearly 80% was in the European Union.",
            "Project examples from 2022 include: EIB support to investments in the water and wastewater infrastructure of the city of Warsaw [...] and support to Andalusia’s rural development programme to improve water catchment, prevent soil erosion.",
       "Beyond the European Union, EIB Global financed the Aqaba-Amman Water Desalination and Conveyance Project, the largest ever investment project for adapting the water sector to the impacts of climate change in Jordan."
        ]
    }
               ]


### Extraction image

In [ ]:

def extract_images_from_pdf(pdf_path, output_folder="images_extraits"):
    """
    Extrait toutes les images d'un fichier PDF et les enregistre dans un dossier "images_extraits".

    Args:
        pdf_path (str): Chemin vers le fichier PDF.
        output_folder (str): Dossier de sortie pour enregistrer les images.

    Returns:
        List[str]: Liste des chemins des images extraites.
    """
    doc = fitz.open(pdf_path)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_paths = []
    image_count = 0

    for page_number in range(len(doc)):
        page = doc[page_number]
        images = page.get_images(full=True)

        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
            image_filename = f"{output_folder}/{pdf_name}_page{page_number + 1}_img{img_index + 1}.{image_ext}"
            with open(image_filename, "wb") as f:
                f.write(image_bytes)

            image_paths.append(image_filename)
            image_count += 1

    print(f"{image_count} image(s) extraite(s) dans le dossier '{output_folder}'.")
    return image_paths


### image pertinente

#### Transcription img

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
def generate_image_caption(image_path):
    """
    Génère une description textuelle (caption) pour une image donnée en utilisant un modèle de type BLIP.

    Args:
        image_path (str): Chemin vers l'image à analyser (doit être un fichier image lisible par PIL).

    Returns:
        str: Légende générée automatiquement décrivant le contenu visuel de l'image.

    """
    raw_image = Image.open(image_path).convert("RGB")
    inputs = processor(raw_image, return_tensors="pt")
    out = model.generate(**inputs, max_new_tokens=200)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption


#### stockage dans un fichier csv

In [ ]:
def caption_dataset(img_paths, output_csv, img_folder="images"):
    """
    Génère un fichier CSV contenant les descriptions (captions) de toutes les images extraites.

    Args:
        img_paths (list): Liste des chemins relatifs des images (fichiers présents dans le dossier `img_folder`).
        output_csv (str): Chemin du fichier CSV à créer pour stocker les résultats.
        img_folder (str): Nom du dossier contenant les images (par défaut "images").

    Returns:
        pd.DataFrame: Un DataFrame contenant deux colonnes :
                      - 'image_filename': le nom du fichier image
                      - 'caption': la description générée automatiquement
    """
    data = []

    for path in img_paths:
        caption = generate_image_caption("images/"+path)
        data.append({
            "image_filename": os.path.basename(path),
            "caption": caption,
        })
        print(f" {os.path.basename(path)} → {img_paths.index(path)} → {caption}")

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False, encoding="utf-8")
    print(f"\n CSV généré avec {len(df)} lignes :::: {output_csv}")
    return df

In [ ]:
img_paths=os.listdir("images/")

In [ ]:
caption_dataset(img_paths,"caption_image.csv")

 20240903-bnpparibas-csr-presentation-2024_page5_img1.png → 0 → a black and white image of a black and white image of a black and white image of a black and
